<a href="https://colab.research.google.com/github/Andyfer004/P1-DS/blob/main/P1_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Universidad del Valle de Guatemala**

## **Facultad de Ingeniería**  
### **Departamento de Ciencias de la Computación**


## **Curso: Data Science**  
### **Profesor: Luis Furlán**


## **Proyecto 1 Data Science: Limpieza de datos**  
### **Autores:**
- **Andy Fuentes 22944**
- **Davis Roldán 22672**
- **Gabriel Paz 221087**


In [ ]:
# Cargar librerías y datos crudos
import pandas as pd

df = pd.read_csv('dataset_institutions.csv')

# Descripción del conjunto de datos crudos
filas, vars_ = df.shape
print(f"Filas: {filas}")
print(f"Variables: {vars_}")

# Vista previa
display(df.head(5))

Filas: 6607
Variables: 17


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ


- El conjunto tiene **`filas`** filas y **`vars_`** variables.  
- No se observan filas totalmente vacías.  
- Algunas columnas muestran tipos mixtos (e.g., números y strings en `TELEFONO`).  
- La vista previa revela formatos inconsistentes (mayúsculas vs minúsculas) en nombres de establecimientos.

## 2. Variables que más operaciones de limpieza necesitarán

A continuación calculamos cuántos valores nulos hay por variable y cuántos valores únicos tiene cada una.


In [ ]:
# Conteo de valores nulos por variable
null_counts = df.isnull().sum().sort_values(ascending=False)
display(null_counts.head(10))

# Conteo de valores únicos por variable
unique_counts = df.nunique().sort_values()
display(unique_counts.head(10))


TELEFONO     53
DIRECTOR     33
DIRECCION     9
SECTOR        7
PLAN          7
JORNADA       7
MODALIDAD     7
STATUS        7
AREA          7
CODIGO        7
dtype: int64

NIVEL              1
STATUS             1
MODALIDAD          2
AREA               3
SECTOR             4
JORNADA            6
PLAN              12
DEPARTAMENTO      23
DEPARTAMENTAL     26
MUNICIPIO        343
dtype: int64

**Top 5 variables con más valores nulos**  
1. `TELEFONO` – 53 nulos  
2. `DIRECTOR` – 33 nulos  
3. `DIRECCION` – 9 nulos  
4. `SECTOR` – 7 nulos  
5. `PLAN` – 7 nulos  

**Otras variables a vigilar** (pocos valores únicos, posibles categorías mal codificadas):  
- `NIVEL`, `STATUS`, `MODALIDAD`, `AREA`, …


## 3. Estrategia de limpieza

A continuación se describe, para cada variable crítica, los pasos de limpieza planeados:

### `ESTABLECIMIENTO`
1. Convertir todo el texto a **minúsculas** (o mayúsculas) para estandarizar.  
2. Eliminar espacios al inicio y final (`.str.strip()`).  
3. Detectar y corregir manualmente o con un diccionario los errores ortográficos más comunes.  
4. Eliminar **duplicados** exactos (`df.drop_duplicates(subset=['ESTABLECIMIENTO'])`).  

### `DIRECCION`
1. `.str.lower()` + `.str.strip()`  
2. Unificar abreviaciones (e.g. “av.” → “avenida”).  
3. Eliminar caracteres no alfanuméricos innecesarios.  
4. Comprobar longitud mínima y marcar flag si está vacía.

### `TELEFONO`
1. Eliminar espacios, guiones y paréntesis:  
   ```python
   df['TELEFONO'] = df['TELEFONO'].str.replace(r'\D+', '', regex=True)
2. Verificar que queden exactamente 8 dígitos (o el formato local).
3. Rellenar con NaN o marcar como “missing” los que no cumplan el formato.

### `SECTOR`
1. .str.lower() → .str.strip().
2. Unificar categorías (“público” vs “publico” vs “Público”).
3. Mapear a un conjunto finito de valores válidos.  

### `DIRECTOR`
1. Convertir a .str.title() para uniformizar nombres.  
2. Eliminar dobles espacios y tildes si es necesario.  
3. Corregir nombres conocidos vía mapeo (map({'J. Pérez': 'Juan Pérez', …})).
4. Revisar manualmente los 33 nulos.

### `PLAN`
1. Eliminar espacios y mayúsculas: .str.upper().str.strip().
2. Validar contra la lista de planes esperados.
3. Revisar manualmente los 7 nulos.